<a href="https://colab.research.google.com/github/loviasol/github-slideshow/blob/master/Copy_of_Copy_of_Copy_of_Copy_of_VQGAN%2BCLIP_An_PUBLIC_COPY.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generate images from text inputs using VQGAN+CLIP (z+quantize method with augmentations).

Notebook created Katherine Crowson (https://github.com/crowsonkb, https://twitter.com/RiversHaveWings). The original BigGAN+CLIP method was created by https://twitter.com/advadnoun. Translation \[to spanish\] modifications and explanations added by Eleiber#8347, user-friendly UI created by Abulafia#3734, translated \[from spanish\] and expanded on by An Graves.

For a detailed tutorial \[in spanish\] on how to use \[the original spanish notebook\], I recommend [visiting this article](https://tuscriaturas.miraheze.org/wiki/Ayuda:Crear_imágenes_con_VQGAN+CLIP), created by Jakeukalane#2767 and Avengium (Ángel)#3715. An English tutorial by \[??? I cannot find who originally posted this and I don't see creator info on the doc itself\] can be found [here](https://docs.google.com/document/d/1Lu7XPRKlNhBQjcKr8k8qRzUzbBW7kzxb5Vu72GMRn2E/edit). These tutorials do not include information on features I (An) have added, which I have hopefully provided adequate instructions for below.


In [ ]:
# @title Licensed under the MIT License

# Copyright (c) 2021 Katherine Crowson

# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to deal
# in the Software without restriction, including without limitation the rights
# to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
# copies of the Software, and to permit persons to whom the Software is
# furnished to do so, subject to the following conditions:

# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.

# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
# OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN
# THE SOFTWARE.

In [ ]:
#@markdown Execute this cell to view the status of the runtime's GPU, otherwise this is not needed.
!nvidia-smi

#Mount Drive (optional)


This notebook supports mounting your google drive as a folder and storing outputs to it in order to keep them when the runtime disconnects. 

If you are running this notebook or a copy of this notebook, this step will require you authorize mounting your drive every time you start up the notebook. I recommend making a new notebook, going to "Runtime->Change Runtime Type" in the menu bar and selecting "GPU" and "High-RAM", and then copying each cell of this notebook in. This way you can mount your drive without manual intervention.

In [ ]:
#@markdown To use your drive, set `root_path` to the relative drive folder path you want outputs to be saved to, then execute the cell. Leaving the field blank or just not running this will have outputs save to the runtime temp storage.
root_path = "" #@param {type: "string"}
abs_root_path = "/content"
if len(root_path) > 0:
    abs_root_path = abs_root_path + "/drive/MyDrive/" + root_path

from google.colab import drive
drive.mount('/content/drive')

#Select and Download Models

In [ ]:
#@markdown By default, the notebook downloads the ImageNet 16384 model. There are others like ImageNet 1024, COCO-Stuff, WikiArt 1024, WikiArt 16384, FacesHQ or S-FLCKR, which are not downloaded by default because it would be a waste of time and space if you don't plan to use them, so if you DO plan to use them, just check those models for download. When your selection is complete, execute this cell.

#@markdown NOTE: Large models like COCO-Stuff can take a long time to download, so if you're losing a lot of time that way and you've mounted your personal Drive above then you may want to copy the .ckpt and .yaml files to somewhere in your drive. If you do this you can easily add your local model to the dropdown (e.g. if your ckpt is at "`drive/MyDrive/colab/coco.ckpt`" the dropdown entry should be "`drive/MyDrive/colab/coco`") and then uncheck the download. If all models you wish to use are local it is fine to uncheck all of these.

imagenet_1024 = False #@param {type:"boolean"}
imagenet_16384 = True #@param {type:"boolean"}
coco = False #@param {type:"boolean"}
faceshq = True #@param {type:"boolean"}
wikiart_1024 = False #@param {type:"boolean"}
wikiart_16384 = True #@param {type:"boolean"}
sflckr = False #@param {type:"boolean"}

if imagenet_1024:
  !curl -L -o vqgan_imagenet_f16_1024.yaml -C - 'http://mirror.io.community/blob/vqgan/vqgan_imagenet_f16_1024.yaml' #ImageNet 1024
  !curl -L -o vqgan_imagenet_f16_1024.ckpt -C - 'http://mirror.io.community/blob/vqgan/vqgan_imagenet_f16_1024.ckpt'  #ImageNet 1024
if imagenet_16384:
  !curl -L -o vqgan_imagenet_f16_16384.yaml -C - 'http://mirror.io.community/blob/vqgan/vqgan_imagenet_f16_16384.yaml' #ImageNet 16384
  !curl -L -o vqgan_imagenet_f16_16384.ckpt -C - 'http://mirror.io.community/blob/vqgan/vqgan_imagenet_f16_16384.ckpt' #ImageNet 16384
if coco:
  !curl -L -o coco.yaml -C - 'https://dl.nmkd.de/ai/clip/coco/coco.yaml' #COCO
  !curl -L -o coco.ckpt -C - 'https://dl.nmkd.de/ai/clip/coco/coco.ckpt' #COCO
if faceshq:
  !curl -L -o faceshq.yaml -C - 'https://drive.google.com/uc?export=download&id=1fHwGx_hnBtC8nsq7hesJvs-Klv-P0gzT' #FacesHQ
  !curl -L -o faceshq.ckpt -C - 'https://app.koofr.net/content/links/a04deec9-0c59-4673-8b37-3d696fe63a5d/files/get/last.ckpt?path=%2F2020-11-13T21-41-45_faceshq_transformer%2Fcheckpoints%2Flast.ckpt' #FacesHQ
if wikiart_1024: 
  !curl -L -o wikiart_1024.yaml -C - 'http://mirror.io.community/blob/vqgan/wikiart.yaml' #WikiArt 1024
  !curl -L -o wikiart_1024.ckpt -C - 'http://mirror.io.community/blob/vqgan/wikiart.ckpt' #WikiArt 1024
if wikiart_16384: 
  !curl -L -o wikiart_16384.yaml -C - 'http://mirror.io.community/blob/vqgan/wikiart_16384.yaml' #WikiArt 16384
  !curl -L -o wikiart_16384.ckpt -C - 'http://mirror.io.community/blob/vqgan/wikiart_16384.ckpt' #WikiArt 16384
if sflckr:
  !curl -L -o sflckr.yaml -C - 'https://heibox.uni-heidelberg.de/d/73487ab6e5314cb5adba/files/?p=%2Fconfigs%2F2020-11-09T13-31-51-project.yaml&dl=1' #S-FLCKR
  !curl -L -o sflckr.ckpt -C - 'https://heibox.uni-heidelberg.de/d/73487ab6e5314cb5adba/files/?p=%2Fcheckpoints%2Flast.ckpt&dl=1' #S-FLCKR

#Library Installation

In [ ]:
# @markdown ###Download Libraries
# @markdown This cell may take some time.
 
print("Downloading CLIP...")
!git clone https://github.com/openai/CLIP                 &> /dev/null
 
print("Installing Python AI libraries...")
!git clone https://github.com/CompVis/taming-transformers &> /dev/null
!pip install ftfy regex tqdm omegaconf pytorch-lightning  &> /dev/null
!pip install kornia                                       &> /dev/null
!pip install einops                                       &> /dev/null
!pip install transformers                                 &> /dev/null
 
print("Installing libraries for metadata handling...")
!pip install stegano                                      &> /dev/null
!apt install exempi                                       &> /dev/null
!pip install python-xmp-toolkit                           &> /dev/null
!pip install imgtag                                       &> /dev/null
!pip install pillow==7.1.2                                &> /dev/null
 
print("Installing Python video creation libraries...")
!pip install imageio-ffmpeg &> /dev/null
!mkdir steps
print("Installation finished.")

In [ ]:
# @title Load Libraries and Definitions

import argparse
import math
from pathlib import Path
import sys
import hashlib
from IPython.display import Audio, clear_output
 
sys.path.append('./taming-transformers')
from IPython import display
from base64 import b64encode
from omegaconf import OmegaConf
from PIL import Image
from taming.models import cond_transformer, vqgan
import torch
from torch import nn, optim
from torch.nn import functional as F
from torchvision import transforms
from torchvision.transforms import functional as TF
from tqdm.notebook import tqdm
 
from CLIP import clip
import kornia.augmentation as K
import numpy as np
import imageio
from PIL import ImageFile, Image
from imgtag import ImgTag    # metadatos 
from libxmp import *         # metadatos
import libxmp                # metadatos
from stegano import lsb
import json
import re
ImageFile.LOAD_TRUNCATED_IMAGES = True
 
def sinc(x):
    return torch.where(x != 0, torch.sin(math.pi * x) / (math.pi * x), x.new_ones([]))
 
 
def lanczos(x, a):
    cond = torch.logical_and(-a < x, x < a)
    out = torch.where(cond, sinc(x) * sinc(x/a), x.new_zeros([]))
    return out / out.sum()
 
 
def ramp(ratio, width):
    n = math.ceil(width / ratio + 1)
    out = torch.empty([n])
    cur = 0
    for i in range(out.shape[0]):
        out[i] = cur
        cur += ratio
    return torch.cat([-out[1:].flip([0]), out])[1:-1]
 
 
def resample(input, size, align_corners=True):
    n, c, h, w = input.shape
    dh, dw = size
 
    input = input.view([n * c, 1, h, w])
 
    if dh < h:
        kernel_h = lanczos(ramp(dh / h, 2), 2).to(input.device, input.dtype)
        pad_h = (kernel_h.shape[0] - 1) // 2
        input = F.pad(input, (0, 0, pad_h, pad_h), 'reflect')
        input = F.conv2d(input, kernel_h[None, None, :, None])
 
    if dw < w:
        kernel_w = lanczos(ramp(dw / w, 2), 2).to(input.device, input.dtype)
        pad_w = (kernel_w.shape[0] - 1) // 2
        input = F.pad(input, (pad_w, pad_w, 0, 0), 'reflect')
        input = F.conv2d(input, kernel_w[None, None, None, :])
 
    input = input.view([n, c, h, w])
    return F.interpolate(input, size, mode='bicubic', align_corners=align_corners)
 
 
class ReplaceGrad(torch.autograd.Function):
    @staticmethod
    def forward(ctx, x_forward, x_backward):
        ctx.shape = x_backward.shape
        return x_forward
 
    @staticmethod
    def backward(ctx, grad_in):
        return None, grad_in.sum_to_size(ctx.shape)
 
 
replace_grad = ReplaceGrad.apply
 
 
class ClampWithGrad(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input, min, max):
        ctx.min = min
        ctx.max = max
        ctx.save_for_backward(input)
        return input.clamp(min, max)
 
    @staticmethod
    def backward(ctx, grad_in):
        input, = ctx.saved_tensors
        return grad_in * (grad_in * (input - input.clamp(ctx.min, ctx.max)) >= 0), None, None
 
 
clamp_with_grad = ClampWithGrad.apply
 
 
def vector_quantize(x, codebook):
    d = x.pow(2).sum(dim=-1, keepdim=True) + codebook.pow(2).sum(dim=1) - 2 * x @ codebook.T
    indices = d.argmin(-1)
    x_q = F.one_hot(indices, codebook.shape[0]).to(d.dtype) @ codebook
    return replace_grad(x_q, x)
 
 
class Prompt(nn.Module):
    def __init__(self, embed, weight=1., stop=float('-inf')):
        super().__init__()
        self.register_buffer('embed', embed)
        self.register_buffer('weight', torch.as_tensor(weight))
        self.register_buffer('stop', torch.as_tensor(stop))
 
    def forward(self, input):
        input_normed = F.normalize(input.unsqueeze(1), dim=2)
        embed_normed = F.normalize(self.embed.unsqueeze(0), dim=2)
        dists = input_normed.sub(embed_normed).norm(dim=2).div(2).arcsin().pow(2).mul(2)
        dists = dists * self.weight.sign()
        return self.weight.abs() * replace_grad(dists, torch.maximum(dists, self.stop)).mean()
 
 
def parse_prompt(prompt):
    vals = prompt.rsplit(':', 2)
    vals = vals + ['', '1', '-inf'][len(vals):]
    return vals[0], float(vals[1]), float(vals[2])
 
 
class MakeCutouts(nn.Module):
    def __init__(self, cut_size, cutn, cut_pow=1.):
        super().__init__()
        self.cut_size = cut_size
        self.cutn = cutn
        self.cut_pow = cut_pow
        self.augs = nn.Sequential(
            K.RandomHorizontalFlip(p=0.5),
            # K.RandomSolarize(0.01, 0.01, p=0.7),
            K.RandomSharpness(0.3,p=0.4),
            K.RandomAffine(degrees=30, translate=0.1, p=0.8, padding_mode='border'),
            K.RandomPerspective(0.2,p=0.4),
            K.ColorJitter(hue=0.01, saturation=0.01, p=0.7))
        self.noise_fac = 0.1
 
 
    def forward(self, input):
        sideY, sideX = input.shape[2:4]
        max_size = min(sideX, sideY)
        min_size = min(sideX, sideY, self.cut_size)
        cutouts = []
        for _ in range(self.cutn):
            size = int(torch.rand([])**self.cut_pow * (max_size - min_size) + min_size)
            offsetx = torch.randint(0, sideX - size + 1, ())
            offsety = torch.randint(0, sideY - size + 1, ())
            cutout = input[:, :, offsety:offsety + size, offsetx:offsetx + size]
            cutouts.append(resample(cutout, (self.cut_size, self.cut_size)))
        batch = self.augs(torch.cat(cutouts, dim=0))
        if self.noise_fac:
            facs = batch.new_empty([self.cutn, 1, 1, 1]).uniform_(0, self.noise_fac)
            batch = batch + facs * torch.randn_like(batch)
        return batch
 
 
def load_vqgan_model(config_path, checkpoint_path):
    config = OmegaConf.load(config_path)
    if config.model.target == 'taming.models.vqgan.VQModel':
        model = vqgan.VQModel(**config.model.params)
        model.eval().requires_grad_(False)
        model.init_from_ckpt(checkpoint_path)
    elif config.model.target == 'taming.models.cond_transformer.Net2NetTransformer':
        parent_model = cond_transformer.Net2NetTransformer(**config.model.params)
        parent_model.eval().requires_grad_(False)
        parent_model.init_from_ckpt(checkpoint_path)
        model = parent_model.first_stage_model
    else:
        raise ValueError(f'unknown model type: {config.model.target}')
    del model.loss
    return model
 
 
def resize_image(image, out_size):
    ratio = image.size[0] / image.size[1]
    area = min(image.size[0] * image.size[1], out_size[0] * out_size[1])
    size = round((area * ratio)**0.5), round((area / ratio)**0.5)
    return image.resize(size, Image.LANCZOS)

#How-To

##Execution Instructions:

The first thing you'll want to change is `prompts:` - here you can put the prompt or prompts you want to generate (separated by `|` ). It's a list because you can use more than one prompt, and the AI will try to 'mix' the images, giving the same weight to each one.

To use a starting image with the model, all you have to do is drop the image file into Colab's file structure (in the left side bar, click the folder if it is hidden), and then change `init_image:` to the exact relative path of the file from the top level shown, e.g. `sample.png` for a file dropped in the top level or `path/to/file/sample.png` if you've placed it in a folder.

You can also change which model to use by changing the line that says `model`. Currently the available models are 1024, 16384, WikiArt, S-FLCKR and COCO-Stuff. To activate them you first must download them above, then just select them from the dropdown. 

You can also use `target_images`, which basically loads one or more images that the AI takes as "meta", serving the same function as a text prompt. To use more than one you must use `|` as a separator.

##Execution Modes

Classic Single Run
> If you leave `run_batch` unchecked and `seed_type` set to "`seed_single`", the program will generate one image using the provided parameters, disregarding `seed_list` and `batch_seed_count`

Random Seed Batch
> If you check `run_batch` and leave `seed_type` set to "`seed_single`", the program will generate `batch_seed_count` many images with the provided parameters except random seeds for each one

Specific Seed Batch
> If you set `seed_type` to "`seed_list`", then the program will execute one 'classic' execution using each seed in the comma-separated list you enter into `seed_list`, REGARDLESS of whether `run_batch` is checked

##Additive Execution

If you set `init_image` to the absolute path of an image already generated by this program (which can be obtained by finding the output in the file broswer on the left side bar, right-clicking, and selecting "Copy Path"), the file name will be parsed to determine if it has the same prompt, seed, and step size as the current run. If it does, then the output will be saved with an iteration count reflecting both the new iterations and those that went into the source image (e.g., run a 500 iteration execution with the output of a 1000 iteration execution as the starting point and it will automatically be saved as the 1500 iteration version of that series).

#Parameters

In [ ]:
prompts = "Tarot Card The Emperor Surrealism \"" #@param {type:"string"}
width =  480#@param {type:"number"}
height =  480#@param {type:"number"}
model_id = "vqgan_imagenet_f16_16384" #@param ["vqgan_imagenet_f16_16384", "vqgan_imagenet_f16_1024", "wikiart_1024", "wikiart_16384", "coco", "drive/MyDrive/colab/coco", "faceshq", "sflckr"]
image_interval =  50#@param {type:"number"}
init_image = "None"#@param {type:"string"}
target_images = None#@param {type:"string"}
#@markdown
seed_single = "4"#@param {type:"string"}
seed_list = "1,2,3" #@param {type:"string"}
seed_type = seed_single #@param ["seed_single", "seed_list"] {type:"raw"}
#@markdown
max_iterations = 500#@param {type:"number"}
run_batch = False#@param {type:"boolean"}
batch_seed_count = 3#@param {type:"number"}
input_images = ""
step_size = 0.1 #@param {type:"slider", min:0, max:1, step:0.05}

model_names={"vqgan_imagenet_f16_16384": 'ImageNet 16384',"vqgan_imagenet_f16_1024":"ImageNet 1024", 
                 "wikiart_1024":"WikiArt 1024", "wikiart_16384":"WikiArt 16384", "coco":"COCO-Stuff", 
                 "drive/MyDrive/colab/coco":"COCO-Stuff (Local)","faceshq":"FacesHQ", "sflckr":"S-FLCKR"}
model_name = model_names[model_id]     

if not 'abs_root_path' in globals():
    abs_root_path = "/content"

if init_image == "None":
    init_image = None
if target_images == "None" or not target_images:
    target_images = []
else:
    target_images = target_images.split("|")
    target_images = [image.strip() for image in target_images]

if init_image or target_images != []:
    input_images = True

prompts_str = prompts
prompt_hash = hashlib.md5(prompts.encode())
prompt_md5 = prompt_hash.hexdigest()

prompts = [frase.strip() for frase in prompts.split("|")]
if prompts == ['']:
    prompts = []

if seed_type == seed_single:
    seed = None if seed_single == -1 else seed_single  
else:
    seed_list = ''.join(seed_list)
    seed = seed_list.replace(" ", "").split(',')

args = argparse.Namespace(
    prompts=prompts,
    image_prompts=target_images,
    noise_prompt_seeds=[],
    noise_prompt_weights=[],
    size=[width, height],
    init_image=init_image,
    init_weight=0.,
    clip_model='ViT-B/32',
    vqgan_config=f'{model_id}.yaml',
    vqgan_checkpoint=f'{model_id}.ckpt',
    step_size=step_size,
    cutn=64,
    cut_pow=1.,
    display_freq=image_interval,
    seed=seed,
    batch_seed_count=(batch_seed_count if run_batch else 1),
    hash = prompt_md5,
    prompt_str = prompts_str,
)

print("Prompt Hash: ", args.hash)

!mkdir {abs_root_path}/{args.hash}
!rm -f {abs_root_path}/{args.hash}/prompt.txt
!echo "{prompts_str}" > {abs_root_path}/{args.hash}/prompt.txt

#Load Model

In [ ]:
#@markdown This cell must be run after parameters are configured, but does NOT need to be run again when parameters are changed

#@markdown Once this has been run successfully you only need to run parameters and then the program to execute with new parameters
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = load_vqgan_model(args.vqgan_config, args.vqgan_checkpoint).to(device)
perceptor = clip.load(args.clip_model, jit=False)[0].eval().requires_grad_(False).to(device)

cut_size = perceptor.visual.input_resolution
e_dim = model.quantize.e_dim
f = 2**(model.decoder.num_resolutions - 1)
make_cutouts = MakeCutouts(cut_size, args.cutn, cut_pow=args.cut_pow)
n_toks = model.quantize.n_e
toksX, toksY = args.size[0] // f, args.size[1] // f
sideX, sideY = toksX * f, toksY * f
z_min = model.quantize.embedding.weight.min(dim=0).values[None, :, None, None]
z_max = model.quantize.embedding.weight.max(dim=0).values[None, :, None, None]

normalize = transforms.Normalize(mean=[0.48145466, 0.4578275, 0.40821073],
                                std=[0.26862954, 0.26130258, 0.27577711])

#Execute the Program

In [ ]:
#@title Execute

def synth(z):
    z_q = vector_quantize(z.movedim(1, 3), model.quantize.embedding.weight).movedim(3, 1)
    return clamp_with_grad(model.decode(z_q).add(1).div(2), 0, 1)

def add_xmp_data(nombrefichero):
    imagen = ImgTag(filename=nombrefichero)
    imagen.xmp.append_array_item(libxmp.consts.XMP_NS_DC, 'creator', 'VQGAN+CLIP', {"prop_array_is_ordered":True, "prop_value_is_array":True})
    if args.prompts:
        imagen.xmp.append_array_item(libxmp.consts.XMP_NS_DC, 'title', " | ".join(args.prompts), {"prop_array_is_ordered":True, "prop_value_is_array":True})
    else:
        imagen.xmp.append_array_item(libxmp.consts.XMP_NS_DC, 'title', 'None', {"prop_array_is_ordered":True, "prop_value_is_array":True})
    imagen.xmp.append_array_item(libxmp.consts.XMP_NS_DC, 'i', str(i), {"prop_array_is_ordered":True, "prop_value_is_array":True})
    imagen.xmp.append_array_item(libxmp.consts.XMP_NS_DC, 'model', model_name, {"prop_array_is_ordered":True, "prop_value_is_array":True})
    imagen.xmp.append_array_item(libxmp.consts.XMP_NS_DC, 'seed',str(seed) , {"prop_array_is_ordered":True, "prop_value_is_array":True})
    imagen.xmp.append_array_item(libxmp.consts.XMP_NS_DC, 'input_images',str(input_images) , {"prop_array_is_ordered":True, "prop_value_is_array":True})
    #for frases in args.prompts:
    #    imagen.xmp.append_array_item(libxmp.consts.XMP_NS_DC, 'Prompt' ,frases, {"prop_array_is_ordered":True, "prop_value_is_array":True})
    imagen.close()

def add_stegano_data(filename):
    data = {
        "title": " | ".join(args.prompts) if args.prompts else None,
        "notebook": "VQGAN+CLIP",
        "i": i,
        "model": model_name,
        "seed": str(seed),
        "input_images": input_images
    }
    lsb.hide(filename, json.dumps(data)).save(filename)

@torch.no_grad()
def checkin(i, losses):
    losses_str = ', '.join(f'{loss.item():g}' for loss in losses)
    tqdm.write(f'i: {i}, loss: {sum(losses).item():g}, losses: {losses_str}')
    out = synth(z)
    TF.to_pil_image(out[0].cpu()).save('progress.png')
    #no one is looking at this info just save some cycles
    #add_stegano_data('progress.png')
    #add_xmp_data('progress.png')
    display.display(display.Image('progress.png'))

def ascend_txt():
    global i
    global z

    out = synth(z)
    iii = perceptor.encode_image(normalize(make_cutouts(out))).float()

    result = []

    if args.init_weight:
        result.append(F.mse_loss(z, z_orig) * args.init_weight / 2)

    for prompt in pMs:
        result.append(prompt(iii))
    img = np.array(out.mul(255).clamp(0, 255)[0].cpu().detach().numpy().astype(np.uint8))[:,:,:]
    img = np.transpose(img, (1, 2, 0))
    filename = f"steps/{i:04}.png"
    imageio.imwrite(filename, np.array(img))
    add_stegano_data(filename)
    add_xmp_data(filename)
    return result

def train(i):
    global opt
    global z

    opt.zero_grad()
    lossAll = ascend_txt()
    if i % args.display_freq == 0:
        checkin(i, lossAll)
    loss = sum(lossAll)
    loss.backward()
    opt.step()
    with torch.no_grad():
        z.copy_(z.maximum(z_min).minimum(z_max))

def make_final_name(s):
    stepstr = "_s{s}".format(s=str(int(args.step_size*100)).zfill(3))
    noiterstr = "{path}/{hash}/{seed}{step}_".format(path=abs_root_path, hash=args.hash, seed=str(s), step=stepstr)
    expr = noiterstr + "(?P<iter>\\d{4})[.]png"
    print(expr)
    newiter = max_iterations
    if(args.init_image is not None):
        if(re.fullmatch(expr, args.init_image) is not None):
            m = re.match(expr, args.init_image)
            iter = m.group("iter")
            inum = int(iter)
            newiter = inum+newiter
    return noiterstr+str(str(newiter).zfill(4))+".png"

def save_final(s):
    out = synth(z)
    img = np.array(out.mul(255).clamp(0, 255)[0].cpu().detach().numpy().astype(np.uint8))[:,:,:]
    img = np.transpose(img, (1, 2, 0))
    filename = make_final_name(s)
    imageio.imwrite(filename, np.array(img))
    add_stegano_data(filename)
    add_xmp_data(filename)

def run_seed():
    torch.cuda.empty_cache()
    global opt
    global z
    global i

    if args.init_image:
        pil_image = Image.open(args.init_image).convert('RGB')
        pil_image = pil_image.resize((sideX, sideY), Image.LANCZOS)
        z, *_ = model.encode(TF.to_tensor(pil_image).to(device).unsqueeze(0) * 2 - 1)
    else:
        one_hot = F.one_hot(torch.randint(n_toks, [toksY * toksX], device=device), n_toks).float()
        z = one_hot @ model.quantize.embedding.weight
        z = z.view([-1, toksY, toksX, e_dim]).permute(0, 3, 1, 2)
    z_orig = z.clone()
    z.requires_grad_(True)
    opt = optim.Adam([z], lr=args.step_size)
    i = 0
    with tqdm() as pbar:
        while True:
            train(i)
            if i == max_iterations:
                break
            i += 1
            pbar.update()

print('Using device:', device)
if prompts:
    print('Using texts:', prompts)

if target_images:
    print('Using image prompts:', target_images)

if args.batch_seed_count is None:
    batch_seed_count = 10
else:
    batch_seed_count = args.batch_seed_count

pMs = []

for prompt in args.prompts:
    txt, weight, stop = parse_prompt(prompt)
    embed = perceptor.encode_text(clip.tokenize(txt).to(device)).float()
    pMs.append(Prompt(embed, weight, stop).to(device))

for prompt in args.image_prompts:
    path, weight, stop = parse_prompt(prompt)
    img = resize_image(Image.open(path).convert('RGB'), (sideX, sideY))
    batch = make_cutouts(TF.to_tensor(img).unsqueeze(0).to(device))
    embed = perceptor.encode_image(normalize(batch)).float()
    pMs.append(Prompt(embed, weight, stop).to(device))

for seed, weight in zip(args.noise_prompt_seeds, args.noise_prompt_weights):
    gen = torch.Generator().manual_seed(seed)
    embed = torch.empty([1, perceptor.visual.output_dim]).normal_(generator=gen)
    pMs.append(Prompt(embed, weight).to(device))

try:
    if type(seed) is list:
        for s in seed:
            clear_output(wait=True)
            torch.manual_seed(s)
            print('Using seed:', s)
            run_seed()
            save_final(s)
    else:
        while batch_seed_count > 0: #unchecking run_batch just forces the count to be 1
            clear_output(wait=True)
            if run_batch or (args.seed is None) or (int(args.seed) < 0):
                s = torch.seed()
            else:
                s = args.seed
            torch.manual_seed(s)
            print('Using seed:', s)
            run_seed()
            save_final(s)
            batch_seed_count -= 1

except KeyboardInterrupt:
    del z
    del z_max
    del z_min
    del opt
    pass


# Generate a Video from the Results

**NOTE: CODE BELOW THIS LINE IS NOT UPDATED FOR BATCH FUNCTIONALITY. Running it will only generate a video of the last image!**

If you want to create a video from the frames, just click below. You can change the frames per second (FPS), the first and last frame, etc.

In [ ]:
#@title Generate
#@markdown `interval` specifies how frequently a frame should be added to the video - 1 is all frames, 2 every other frame, etc.
first_frame = 1 #@param {type: Number} #Este es el frame donde el vídeo empezará
last_frame = 500 #@param {type: Number} #Puedes cambiar i a el número del último frame que quieres generar. It will raise an error if that number of frames does not exist.

fps = 50 #@param {type: Number} #
interval = 1 #@param {type: Number} #

total_frames = last_frame-first_frame

frames = []
tqdm.write('Generando video...')

#ensure first frame is always appended
if(first_frame % interval is not 0):
    filename = f"steps/{first_frame:04}.png"
    frames.append(Image.open(filename))

for i in range(first_frame,min(last_frame,max_iterations)+1): #
    if(i % interval is not 0):
       continue
    filename = f"steps/{i:04}.png"
    frames.append(Image.open(filename))

#ensure last frame is always appended
if(last_frame % interval is not 0):
    filename = f"steps/{last_frame:04}.png"
    frames.append(Image.open(filename))

from subprocess import Popen, PIPE
p = Popen(['ffmpeg', '-y', '-f', 'image2pipe', '-vcodec', 'png', '-r', str(fps), '-i', '-', '-vcodec', 'libx264', '-r', str(fps), '-pix_fmt', 'yuv420p', '-crf', '17', '-preset', 'veryslow', 'video.mp4'], stdin=PIPE)
for im in tqdm(frames):
    im.save(p.stdin, 'PNG')
p.stdin.close()

print("The video is being compiled, please wait...")
p.wait()
print("The video is ready.")

In [ ]:
# @title Download Video
from google.colab import files
files.download("video.mp4")

In [ ]:
# @title View the Video in Browser
# @markdown This process can be a bit slow. If you don't want to wait, download it by executing the next cell instead of this one.
mp4 = open('video.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
display.HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)